In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql

In [3]:
world = pymysql.connect(user='root', \
      password='cheetor', \
      host= '127.0.0.1', \
      port=3306, \
      db='world', \
      autocommit=True)


In [74]:
country = pd.read_sql_query('select * from Country', con=world,index_col='Code')
city = pd.read_sql_query('select * from city', con=world,index_col='ID')
language = pd.read_sql_query('select * from countrylanguage', con=world,index_col=['CountryCode','Language']).reset_index()

1.select  *from country where population > 50000000 order by population DESC limit 10


In [14]:
country[(country.Population>50000000)].sort_values(by=['Population'],ascending=False).head(10)

,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
Code,,,,,,,,,,,,,,
CHN,China,Asia,Eastern Asia,9572900.0,-1523.0,1277558000,71.4,982268.0,917719.0,Zhongquo,People'sRepublic,Jiang Zemin,1891.0,CN
IND,India,Asia,Southern and Central Asia,3287263.0,1947.0,1013662000,62.5,447114.0,430572.0,Bharat/India,Federal Republic,Kocheril Raman Narayanan,1109.0,IN
USA,United States,North America,North America,9363520.0,1776.0,278357000,77.1,8510700.0,8110900.0,United States,Federal Republic,George W. Bush,3813.0,US
IDN,Indonesia,Asia,Southeast Asia,1904569.0,1945.0,212107000,68.0,84982.0,215002.0,Indonesia,Republic,Abdurrahman Wahid,939.0,ID
BRA,Brazil,South America,South America,8547403.0,1822.0,170115000,62.9,776739.0,804108.0,Brasil,Federal Republic,Fernando Henrique Cardoso,211.0,BR
PAK,Pakistan,Asia,Southern and Central Asia,796095.0,1947.0,156483000,61.1,61289.0,58549.0,Pakistan,Republic,Mohammad Rafiq Tarar,2831.0,PK
RUS,Russian Federation,Europe,Eastern Europe,17075400.0,1991.0,146934000,67.2,276608.0,442989.0,Rossija,Federal Republic,Vladimir Putin,3580.0,RU
BGD,Bangladesh,Asia,Southern and Central Asia,143998.0,1971.0,129155000,60.2,32852.0,31966.0,Bangladesh,Republic,Shahabuddin Ahmad,150.0,BD
JPN,Japan,Asia,Eastern Asia,377829.0,-660.0,126714000,80.7,3787042.0,4192638.0,Nihon/Nippon,Constitutional Monarchy,Akihito,1532.0,JP


2. select Continent, count(*) As Number_Countries, sum(population) As Population
from country
where population > 0
group by Continent
order by 1 ASC

In [86]:
country['Number_Countries'] = 1
country[['Continent','Population','Number_Countries']].where(country.Population > 0).groupby('Continent').sum()

,Population,Number_Countries
Continent,,
Africa,7.844750e+08,57.0
Asia,3.705026e+09,51.0
Europe,7.300746e+08,46.0
North America,4.829930e+08,37.0
Oceania,3.040115e+07,27.0
South America,3.457800e+08,14.0


3. select city.Name As City, city.population
from city
inner join country ON city.CountryCode = country.code
where country.code = 'USA'
order by city.population DESC limit 10

In [66]:
citycountry = pd.merge(city,country, how = 'inner',left_on='CountryCode',right_index = True,suffixes = ('_city','_country'))
citycountry = citycountry.rename(columns = {'Name_city':'City','Population_city':'population'})
citycountry[['City','population']].where(citycountry.CountryCode=='USA').groupby('City').sum().sort_values(by=['population'],ascending=False).head(10)

,population
City,
New York,8008278.0
Los Angeles,3694820.0
Chicago,2896016.0
Houston,1953631.0
Philadelphia,1517550.0
Phoenix,1321045.0
San Diego,1223400.0
Dallas,1188580.0
San Antonio,1144646.0


4. select country.Name, Language, (Percentage * population) / 100
from countrylanguage
inner join country on countrylanguage.CountryCode = country.code
where IsOfficial = True
order by 3 DESC limit 10

In [87]:
languagecountry = pd.merge(language,country, how = 'inner',left_on='CountryCode',right_index = True,suffixes = ('_language','_country'))
lcOfficial = languagecountry.where(languagecountry.IsOfficial =='T')
lcOfficial['Speaking Population'] = lcOfficial['Percentage']*lcOfficial['Population']/100
lcOfficial[['Name','Language','Speaking Population']].sort_values(by=['Speaking Population'], ascending = False).head(10)

,Name,Language,Speaking Population
159,China,Chinese,1.175353e+09
397,India,Hindi,4.044511e+08
911,United States,English,2.399437e+08
116,Brazil,Portuguese,1.658621e+08
755,Russian Federation,Russian,1.272448e+08
83,Bangladesh,Bengali,1.261844e+08
444,Japan,Japanese,1.255736e+08
548,Mexico,Spanish,9.106940e+07
232,Germany,German,7.501637e+07
946,Vietnam,Vietnamese,6.929418e+07


5. select Language, sum((Percentage * population) / 100)
from countrylanguage
inner join country ON countrylanguage.CountryCode = country.code
group by Language
order by 2 desc limit 5

In [89]:
languagecountry['Speaking Population'] = languagecountry['Percentage']*languagecountry['Population']/100
languagecountry[['Language','Speaking Population']].groupby('Language').sum().sort_values(by=['Speaking Population'], ascending = False).head(5)

,Speaking Population
Language,
Chinese,1.191844e+09
Hindi,4.056331e+08
Spanish,3.550295e+08
English,3.470779e+08
Arabic,2.338392e+08
